In [ ]:
from hcmus.utils import data_utils, transform_utils

splits = data_utils.get_data_splits()
transform_train, transform_test = transform_utils.get_transforms_downscale_transfer_learning()
datasets = data_utils.get_image_datasets_v2(splits, transform_train, transform_test, random_margin=0.2)
dataloaders = data_utils.get_data_loaders_v2(datasets, {
    "train": True
})

2025-07-10 10:05:52.574 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 3443 tasks available.
2025-07-10 10:05:52.575 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 35
Loading tasks: 100%|██████████| 35/35 [00:09<00:00,  3.55it/s]
2025-07-10 10:06:09.224 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 418 tasks available.
2025-07-10 10:06:09.224 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 5
Loading tasks: 100%|██████████| 5/5 [00:03<00:00,  1.49it/s]
2025-07-10 10:06:15.820 | WARNING  | hcmus.lbs._label_studio_connector:get_tasks:148 - Page size is too large, only 286 tasks available.
2025-07-10 10:06:15.821 | INFO     | hcmus.lbs._label_studio_connector:get_tasks:152 - New `page_to` applied: 3
Loading tasks:  67%|██████▋   | 2/3 [00:01<00:00,  1.38it/s]

In [ ]:
from hcmus.models.backbone import CLIPBackbone
from hcmus.models.backbone import DinoBackbone
from hcmus.models.transfer import TransferNetwork
from hcmus.models.transfer import TransferTrainer

In [ ]:
backbone_list = [
    # (DinoBackbone, {"model_id": "facebook/dinov2-small"}),
    # (DinoBackbone, {"model_id": "facebook/dinov2-base"}),
    (DinoBackbone, {"model_id": "facebook/dino-vitb8"}),
    (DinoBackbone, {"model_id": "facebook/dino-vits8"}),
    (DinoBackbone, {"model_id": "facebook/dino-vits16"}),
    (DinoBackbone, {"model_id": "facebook/dino-vitb16"}),
    (CLIPBackbone, {"backbone_name": "ViT-B/32"}),
    (CLIPBackbone, {"backbone_name": "ViT-B/16"}),
]

In [ ]:
import mlflow
def get_or_create_experiment():
    name = "/TransferLearning"
    try:
        mlflow.create_experiment(name)
    except:
        pass

    return mlflow.get_experiment_by_name(name).experiment_id

In [ ]:
from hcmus.utils import mlflow_utils
get_or_create_experiment()
for cls, params in backbone_list:
    mlflow.end_run()
    backbone = cls(**params)
    model = TransferNetwork(backbone, output_dim=99)
    trainer = TransferTrainer(
        model=model,
        mlflow_experiment_name="/TransferLearning",
        mlflow_run_name=cls.__name__ + f"/{params}"
    )
    trainer.train(32, dataloaders["train"], dataloaders["val"])
    trainer.load_model("best_model.pt")
    trainer.evaluate(dataloaders["test"], log_mlflow=True)
    mlflow_utils.log_json_artifact(datasets["train"].label2idx, "label2idx.json")
    trainer.end_run()